In [1]:
import requests
from requests import Response
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res:Response = requests.get(youbike_url)

if res.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")


from pydantic import BaseModel,Field,field_validator,RootModel
class Site (BaseModel):
    站名:str = Field(alias='sna') 
    行政區域:str = Field(alias='sarea')
    時間:str = Field(alias='mday')
    位置:str = Field(alias='ar')
    狀態:bool = Field(alias='act')
    總數量:int = Field(alias='total')
    可借:int = Field(alias='available_rent_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')     
    可還:int = Field(alias='available_return_bikes')

    @field_validator('站名',mode='before')
    @classmethod
    def get_name(cls,value:str) ->str:
         return value.split('_')[-1]   

class Youbick(RootModel):
    root:list[Site]


data = Youbick.model_validate_json(res.text)
all_sites = data.model_dump()


import pandas as pd
df = pd.DataFrame(all_sites)
df


下載成功


,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-24 13:10:55,復興南路二段235號前,True,28,2,25.02605,121.54360,26
1,復興南路二段273號前,大安區,2024-06-24 13:10:29,復興南路二段273號西側,True,21,1,25.02565,121.54357,20
2,國北教大實小東側門,大安區,2024-06-24 13:08:23,和平東路二段96巷7號,True,16,11,25.02429,121.54124,5
3,和平公園東側,大安區,2024-06-24 13:11:02,和平東路二段118巷33號,True,11,6,25.02351,121.54282,5
4,辛亥復興路口西北側,大安區,2024-06-24 13:11:21,復興南路二段368號,True,16,3,25.02153,121.54299,13
...,...,...,...,...,...,...,...,...,...,...
1424,臺大總圖書館西南側,臺大公館校區,2024-06-24 13:10:55,臺大圖書館西南側,True,30,20,25.01690,121.54031,1
1425,臺大黑森林西側,臺大公館校區,2024-06-24 13:10:29,臺大霖澤館南側,True,20,19,25.01995,121.54347,1
1426,臺大獸醫館南側,臺大公館校區,2024-06-24 13:10:23,臺大獸醫系館南側,True,24,22,25.01791,121.54242,0
1427,臺大新體育館東南側,臺大公館校區,2024-06-24 13:09:21,臺大體育館東側,True,40,21,25.02112,121.53591,6


In [3]:
df[df['可借'] <= 3]

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-24 13:10:55,復興南路二段235號前,True,28,2,25.02605,121.54360,26
1,復興南路二段273號前,大安區,2024-06-24 13:10:29,復興南路二段273號西側,True,21,1,25.02565,121.54357,20
4,辛亥復興路口西北側,大安區,2024-06-24 13:11:21,復興南路二段368號,True,16,3,25.02153,121.54299,13
6,復興南路二段340巷口,大安區,2024-06-24 13:11:02,復興南路二段342號,True,11,2,25.02253,121.54326,9
7,新生南路三段52號前,大安區,2024-06-24 13:06:21,新生南路三段52號,True,17,2,25.02112,121.53407,15
...,...,...,...,...,...,...,...,...,...,...
1411,臺大生命科學館西北側,臺大公館校區,2024-06-24 13:03:20,臺大動物博物館西側,True,30,1,25.01548,121.53825,29
1413,臺大博理館西側,臺大公館校區,2024-06-24 13:08:23,臺大博理館西側,True,22,3,25.01922,121.54195,9
1416,臺大一號館,臺大公館校區,2024-06-24 13:06:15,臺大植物魚類標本館一號館北側,True,19,0,25.01698,121.53455,19
1417,臺大小小福西南側,臺大公館校區,2024-06-24 13:10:23,臺大進修教育大樓,True,10,0,25.01539,121.53686,10


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1429 entries, 0 to 1428
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1429 non-null   object 
 1   行政區域    1429 non-null   object 
 2   時間      1429 non-null   object 
 3   位置      1429 non-null   object 
 4   狀態      1429 non-null   bool   
 5   總數量     1429 non-null   int64  
 6   可借      1429 non-null   int64  
 7   緯度      1429 non-null   float64
 8   經度      1429 non-null   float64
 9   可還      1429 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 102.0+ KB


In [5]:
import numpy as np

less3_df = df.query('可借<=3')
less3_df.shape

(441, 10)

In [6]:
df2 = df.query('可借<=3 and 可還>=30')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
45,臥龍樂業街口,大安區,2024-06-24 13:01:15,臥龍街234-2號,True,41,0,25.01856,121.55558,41
77,建國濟南路口,大安區,2024-06-24 13:04:21,建國南路一段/濟南路三段口(東南側),True,54,3,25.03935,121.53828,51
93,建國南路一段279巷口,大安區,2024-06-24 13:10:55,建國南路一段275號,True,32,0,25.03635,121.53833,31
95,捷運忠孝新生站(4號出口),大安區,2024-06-24 13:09:21,忠孝新生路口北側,True,34,1,25.04239,121.53324,33
104,捷運忠孝復興站(5號出口),大安區,2024-01-14 04:19:10,復興南路一段135巷3號(對面),False,41,0,25.04262,121.54443,41
124,捷運公館站(3號出口),大安區,2024-06-24 13:10:29,捷運公館站(3號出口)西側,True,99,0,25.01551,121.53374,99
126,忠孝東路三段217巷口,大安區,2024-06-06 15:05:16,忠孝東路三段229-1號前,False,34,0,25.04176,121.54027,34
148,捷運大安站(3號出口),大安區,2024-06-24 13:10:29,信義路三段184號前,True,46,3,25.03328,121.54250,43
149,1,大安區,2024-06-24 13:10:28,羅斯福路三段201號旁,True,38,3,25.02055,121.52855,35
189,重慶酒泉街口,大同區,2024-06-24 13:05:16,重慶北路三段260號前,True,44,0,25.07158,121.51368,44


In [9]:
df2 = df.query('可借<=3 or 可還<=3')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
2,國北教大實小東側門,大安區,2024-06-17 19:26:22,和平東路二段96巷7號,True,16,3,25.02429,121.54124,13
3,和平公園東側,大安區,2024-06-17 19:26:21,和平東路二段118巷33號,True,11,10,25.02351,121.54282,1
5,復興南路二段280號前,大安區,2024-06-17 19:05:23,復興南路二段280號,True,11,10,25.02429,121.54328,1
6,復興南路二段340巷口,大安區,2024-06-17 19:25:22,復興南路二段342號,True,11,8,25.02253,121.54326,3
7,新生南路三段52號前,大安區,2024-06-17 19:17:30,新生南路三段52號,True,17,17,25.02112,121.53407,0
...,...,...,...,...,...,...,...,...,...,...
1416,臺大電機二館東南側,臺大公館校區,2024-06-17 19:25:22,臺大電機二館南側,True,14,3,25.01857,121.54247,11
1417,臺大圖資系館北側,臺大公館校區,2024-06-17 19:25:22,臺大圖書資訊館北側,True,25,1,25.01804,121.53928,24
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:25:22,臺大圖書館西南側,True,30,0,25.01690,121.54031,30
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,True,20,1,25.01995,121.54347,19


In [10]:
df3 = df.query('行政區域=="大安區" and 可借==0')
df3

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.02093,121.53310,5
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,True,18,0,25.01576,121.54291,18
51,法治公園,大安區,2024-06-17 19:26:22,通化街189號西側,True,15,0,25.02694,121.55325,10
67,敦化南路二段293巷口南側,大安區,2024-06-17 18:58:15,敦化南路二段293巷口南側,True,14,0,25.02370,121.54899,14
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,True,38,0,25.02215,121.54839,36
69,安和路二段181巷口,大安區,2024-06-17 19:16:15,安和路二段185號,True,17,0,25.02721,121.54988,17
76,復興南路1段340巷口,大安區,2024-06-17 19:26:17,復興南路一段信義路三段147巷45弄口,True,50,0,25.03701,121.54348,50
88,昌隆公園,大安區,2024-06-17 19:26:22,市民大道三段206號西側,True,16,0,25.04448,121.53946,16
98,瑠公公園,大安區,2024-03-30 08:59:14,忠孝東路四段49巷4弄/大安路口,False,20,0,25.04201,121.54612,20
102,東豐公園,大安區,2024-06-17 19:21:15,東豐街24號(旁公園路側),True,19,0,25.03569,121.54510,19


In [11]:
n = 0
df4 = df.query('行政區域=="大安區" and 可借==@n')
df4

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.02093,121.53310,5
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,True,18,0,25.01576,121.54291,18
51,法治公園,大安區,2024-06-17 19:26:22,通化街189號西側,True,15,0,25.02694,121.55325,10
67,敦化南路二段293巷口南側,大安區,2024-06-17 18:58:15,敦化南路二段293巷口南側,True,14,0,25.02370,121.54899,14
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,True,38,0,25.02215,121.54839,36
69,安和路二段181巷口,大安區,2024-06-17 19:16:15,安和路二段185號,True,17,0,25.02721,121.54988,17
76,復興南路1段340巷口,大安區,2024-06-17 19:26:17,復興南路一段信義路三段147巷45弄口,True,50,0,25.03701,121.54348,50
88,昌隆公園,大安區,2024-06-17 19:26:22,市民大道三段206號西側,True,16,0,25.04448,121.53946,16
98,瑠公公園,大安區,2024-03-30 08:59:14,忠孝東路四段49巷4弄/大安路口,False,20,0,25.04201,121.54612,20
102,東豐公園,大安區,2024-06-17 19:21:15,東豐街24號(旁公園路側),True,19,0,25.03569,121.54510,19
